Hier entsteht das Projekt bla bla bla

In [1]:
import pandas as pd
import numpy as np
import re

In [28]:
player_stats = pd.read_csv('2021-2022_Football_Player_Stats.csv', encoding='latin1', sep=';')
fifa_data = pd.read_csv('FIFA23_official_data.csv', encoding='latin1')

columns_to_remove_fifa = ['Photo', 'Club Logo', 'Flag', 'Real Face', 'Best Overall Rating', 
                         'Joined', 'Loaned From', 'Contract Valid Until', 'Release Clause', 'Kit Number']
fifa_data = fifa_data.drop(columns=[col for col in columns_to_remove_fifa if col in fifa_data.columns])

player_stats = player_stats[player_stats['Min'] >= 100]
player_stats = player_stats[player_stats['MP'] >= 5]

def extract_last_name_from_full(full_name):
    if pd.isna(full_name):
        return ""
    parts = str(full_name).strip().split()
    return parts[-1].lower() if parts else ""

def extract_names_from_fifa(fifa_name):
    if pd.isna(fifa_name):
        return "", ""
    
    name_str = str(fifa_name).strip()
    name_str = re.sub(r'[^a-zA-Z\s\.]', '', name_str)
    parts = name_str.split()
    
    if len(parts) >= 2:
        first_name_abbr = parts[0].replace('.', '').lower()
        last_name = parts[-1].lower()
        return first_name_abbr, last_name
    elif len(parts) == 1:
        return "", parts[0].lower()
    else:
        return "", ""

player_stats['last_name_temp'] = player_stats['Player'].apply(extract_last_name_from_full)
fifa_data['last_name_temp'] = fifa_data['Name'].apply(lambda x: extract_names_from_fifa(x)[1])

common_last_names = set(player_stats['last_name_temp']).intersection(set(fifa_data['last_name_temp']))

def find_best_matches():
    matches = []
    
    for last_name in common_last_names:
        stats_players = player_stats[player_stats['last_name_temp'] == last_name]
        fifa_players = fifa_data[fifa_data['last_name_temp'] == last_name]
        
        for _, stats_row in stats_players.iterrows():
            stats_full_name = stats_row['Player']
            stats_first_initial = stats_full_name[0].lower() if stats_full_name else ""
            
            for _, fifa_row in fifa_players.iterrows():
                fifa_full_name = fifa_row['Name']
                fifa_first_abbr = extract_names_from_fifa(fifa_full_name)[0]
                
                if (stats_first_initial == fifa_first_abbr or 
                    len(stats_players) == 1 or 
                    len(fifa_players) == 1):
                    
                    matches.append({
                        'stats_name': stats_full_name,
                        'fifa_name': fifa_full_name,
                        'last_name': last_name,
                        'stats_index': stats_row.name,
                        'fifa_index': fifa_row.name,
                        'confidence': 'high' if stats_first_initial == fifa_first_abbr else 'medium'
                    })
    
    return matches

best_matches = find_best_matches()

if len(best_matches) > 0:
    stats_indices = [match['stats_index'] for match in best_matches]
    fifa_indices = [match['fifa_index'] for match in best_matches]
    
    player_stats_filtered = player_stats.loc[stats_indices].drop('last_name_temp', axis=1)
    fifa_data_filtered = fifa_data.loc[fifa_indices].drop('last_name_temp', axis=1)

    for i in range(min(3, len(player_stats_filtered))):
        player_name = player_stats_filtered['Player'].iloc[i]
        goals = player_stats_filtered['Goals'].iloc[i] if 'Goals' in player_stats_filtered.columns else 'N/A'
        assists = player_stats_filtered['Assists'].iloc[i] if 'Assists' in player_stats_filtered.columns else 'N/A'
        minutes = player_stats_filtered['Min'].iloc[i] if 'Min' in player_stats_filtered.columns else 'N/A'
        
        fifa_match = fifa_data_filtered[fifa_data_filtered['Name'].str.contains(player_name.split()[-1], na=False)]
        fifa_overall = fifa_match['Overall'].iloc[0] if len(fifa_match) > 0 else 'N/A'
    
    player_stats_filtered.to_csv('player_stats_complete_filtered.csv', index=False)
    fifa_data_filtered.to_csv('fifa_data_complete_filtered.csv', index=False)
    
    mapping_df = pd.DataFrame(best_matches)
    
    combined_data = player_stats_filtered.copy()

In [ ]:
player_stats_filtered = pd.read_csv('player_stats_complete_filtered.csv')
fifa_data_filtered = pd.read_csv('fifa_data_complete_filtered.csv')

fifa_duplicates = fifa_data_filtered[fifa_data_filtered.duplicated(subset=['Name'], keep=False)]

if len(fifa_duplicates) > 0:
    fifa_data_clean = fifa_data_filtered.sort_values('Overall', ascending=False).drop_duplicates(subset=['Name'], keep='first')
else:
    fifa_data_clean = fifa_data_filtered.copy()

player_stats_filtered = player_stats_filtered.copy()
player_stats_filtered['unique_key'] = player_stats_filtered.apply(
    lambda x: f"{x['Player']}_{x['Born']}_{x['Nation']}", 
    axis=1
)

transfer_players = player_stats_filtered[player_stats_filtered.duplicated(subset=['unique_key'], keep=False)]

if len(transfer_players) > 0:    
    for player_key in transfer_players['unique_key'].unique():
        player_entries = transfer_players[transfer_players['unique_key'] == player_key]
    
    player_stats_clean = player_stats_filtered.drop_duplicates(subset=['unique_key'], keep=False)
    player_stats_clean = player_stats_clean.drop('unique_key', axis=1)
else:
    player_stats_clean = player_stats_filtered.drop('unique_key', axis=1)

def get_last_name(name):
    return str(name).split()[-1].lower() if pd.notna(name) else ""

player_stats_clean['last_name_temp'] = player_stats_clean['Player'].apply(get_last_name)
fifa_data_clean['last_name_temp'] = fifa_data_clean['Name'].apply(get_last_name)

common_players_after_clean = set(player_stats_clean['last_name_temp']).intersection(
    set(fifa_data_clean['last_name_temp'])
)

player_stats_final = player_stats_clean[
    player_stats_clean['last_name_temp'].isin(common_players_after_clean)
].drop('last_name_temp', axis=1)

fifa_data_final = fifa_data_clean[
    fifa_data_clean['last_name_temp'].isin(common_players_after_clean)
].drop('last_name_temp', axis=1)

player_stats_final.to_csv('player_stats_cleaned_separate.csv', index=False)

fifa_data_final.to_csv('fifa_data_cleaned_separate.csv', index=False)

mapping_data = []

for stats_idx, stats_row in player_stats_final.iterrows():
    stats_name = stats_row['Player']
    stats_last_name = get_last_name(stats_name)
    
    matching_fifa = fifa_data_final[
        fifa_data_final['Name'].apply(get_last_name) == stats_last_name
    ]
    
    if len(matching_fifa) > 0:
        fifa_name = matching_fifa['Name'].iloc[0]
        fifa_overall = matching_fifa['Overall'].iloc[0]
        
        mapping_data.append({
            'Stats_Player': stats_name,
            'FIFA_Player': fifa_name,
            'Last_Name': stats_last_name,
            'FIFA_Overall': fifa_overall,
            'Stats_Squad': stats_row['Squad'] if 'Squad' in stats_row else 'N/A',
            'FIFA_Club': matching_fifa['Club'].iloc[0] if 'Club' in matching_fifa.columns else 'N/A'
        })

mapping_df = pd.DataFrame(mapping_data)
mapping_df.to_csv('player_mapping_separate.csv', index=False)

🔄 Entferne 422 Spieler mit Transfers:
   ❌ Joselu_1990_ESP - Vereine: ['Alavï¿½s', 'Alavï¿½s']
   ❌ Sï¿½rgio Oliveira_1992_POR - Vereine: ['Roma', 'Roma', 'Roma', 'Roma', 'Roma', 'Roma', 'Roma', 'Roma']
   ❌ Andrea Schiavone_1993_ITA - Vereine: ['Salernitana', 'Salernitana']
   ❌ Fernando_1987_BRA - Vereine: ['Sevilla', 'Sevilla', 'Sevilla', 'Sevilla']
   ❌ Christoph Kramer_1991_GER - Vereine: ["M'Gladbach", "M'Gladbach"]
   ❌ Wataru Endo_1993_JPN - Vereine: ['Stuttgart', 'Stuttgart']
   ❌ Rï¿½ben Dias_1997_POR - Vereine: ['Manchester City', 'Manchester City', 'Manchester City', 'Manchester City']
   ❌ Jonas Hofmann_1992_GER - Vereine: ["M'Gladbach", "M'Gladbach", "M'Gladbach"]
   ❌ Nahuel Molina_1997_ARG - Vereine: ['Udinese', 'Udinese']
   ❌ Kevin Vogt_1991_GER - Vereine: ['Hoffenheim', 'Hoffenheim']
   ❌ Son_1994_ESP - Vereine: ['Levante', 'Levante']
   ❌ Timothy Weah_2000_USA - Vereine: ['Lille', 'Lille']
   ❌ Tolgay Arslan_1990_GER - Vereine: ['Udinese', 'Udinese', 'Udinese']
   ❌

In [30]:
player_stats_cleaned_separate = pd.read_csv('player_stats_cleaned_separate.csv')
player_stats_cleaned_separate

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Off,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%
0,923,Juan Foyth,ARG,DF,Villarreal,La Liga,24.0,1998,25,22,...,0.00,0.47,2.15,0.00,0.05,0.0,7.85,2.24,1.03,68.6
1,97,Robert Andrich,GER,MF,Leverkusen,Bundesliga,27.0,1994,26,21,...,0.10,0.25,1.63,0.00,0.00,0.0,11.30,2.08,1.09,65.6
2,426,Jens Cajuste,SWE,MF,Reims,Ligue 1,22.0,1999,8,4,...,0.00,0.50,1.75,0.00,0.00,0.0,5.75,0.50,1.50,25.0
3,2297,Mads Roerslev,DEN,DF,Brentford,Premier League,22.0,1999,21,12,...,0.14,1.67,0.65,0.00,0.07,0.0,5.94,1.16,2.17,34.8
4,530,Joris Chotard,FRA,MF,Montpellier,Ligue 1,20.0,2001,36,33,...,0.00,0.46,1.01,0.00,0.00,0.0,8.59,0.77,0.89,46.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,2280,Enzo Roco,CHI,DF,Elche,La Liga,29.0,1992,30,29,...,0.04,0.00,0.81,0.00,0.07,0.0,8.13,2.82,1.27,69.0
1074,2605,Jonathan Tah,GER,DF,Leverkusen,Bundesliga,26.0,1996,33,33,...,0.03,0.09,0.56,0.00,0.00,0.0,9.94,3.41,1.28,72.7
1075,2773,Edoardo Vergani,ITA,FWMF,Salernitana,Serie A,21.0,2001,6,1,...,0.91,0.00,0.91,0.00,0.00,0.0,5.45,1.82,2.73,40.0
1076,2182,Teemu Pukki,FIN,FW,Norwich City,Premier League,32.0,1990,37,37,...,0.33,0.69,0.30,0.03,0.00,0.0,3.43,0.44,2.99,12.9


In [31]:
fifa_data_cleaned_separate = pd.read_csv('fifa_data_cleaned_separate.csv')
fifa_data_cleaned_separate

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight
0,165153,K. Benzema,34,France,91,91,Real Madrid CF,â¬64M,â¬450K,2147,Right,4.0,4.0,4.0,Medium/ Medium,Normal (170-185),"<span class=""pos pos21"">CF",185cm,81kg
1,158023,L. Messi,35,Argentina,91,91,Paris Saint-Germain,â¬54M,â¬195K,2190,Left,5.0,4.0,4.0,Low/ Low,Unique,"<span class=""pos pos23"">RW",169cm,67kg
2,192985,K. De Bruyne,31,Belgium,91,91,Manchester City,â¬107.5M,â¬350K,2303,Right,4.0,5.0,4.0,High/ High,Unique,"<span class=""pos pos13"">RCM",181cm,70kg
3,203376,V. van Dijk,30,Netherlands,90,90,Liverpool,â¬98M,â¬230K,2117,Right,4.0,3.0,2.0,Medium/ High,Unique,"<span class=""pos pos6"">LCB",193cm,92kg
4,167495,M. Neuer,36,Germany,90,90,FC Bayern MÃ¼nchen,â¬13.5M,â¬72K,1535,Right,5.0,4.0,1.0,Medium/ Medium,Unique,"<span class=""pos pos0"">GK",193cm,93kg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,256873,21Â J. Paris,18,Germany,52,69,KFC Uerdingen 05,â¬180K,â¬500,824,Right,1.0,2.0,1.0,Medium/ Medium,Normal (185+),"<span class=""pos pos29"">RES",188cm,83kg
1105,262994,22Â B. McLean,18,England,52,60,Burton Albion,â¬160K,â¬500,1428,Right,1.0,2.0,2.0,Medium/ Medium,Lean (170-),"<span class=""pos pos29"">RES",168cm,65kg
1106,265630,22Â B. Williams,16,England,51,64,Harrogate Town,â¬130K,â¬500,1471,Right,1.0,3.0,2.0,Medium/ Medium,Lean (170-185),"<span class=""pos pos28"">SUB",180cm,70kg
1107,262557,M. Caputo,16,Australia,50,70,Melbourne City FC,â¬130K,â¬500,1324,Right,1.0,3.0,2.0,Medium/ Low,Lean (170-185),"<span class=""pos pos28"">SUB",182cm,74kg


In [32]:
player_stats = pd.read_csv('player_stats_cleaned_separate.csv')
fifa_data = pd.read_csv('fifa_data_cleaned_separate.csv')

def get_last_name(name):
    return str(name).split()[-1].lower() if pd.notna(name) else ""

player_stats['last_name_temp'] = player_stats['Player'].apply(get_last_name)
fifa_data['last_name_temp'] = fifa_data['Name'].apply(get_last_name)

def find_strict_matches():
    matches = []
    
    for stats_idx, stats_row in player_stats.iterrows():
        stats_name = stats_row['Player']
        stats_last_name = stats_row['last_name_temp']
        stats_first_initial = stats_name[0].lower() if stats_name else ""
        
        possible_fifa_matches = fifa_data[fifa_data['last_name_temp'] == stats_last_name]
        
        for fifa_idx, fifa_row in possible_fifa_matches.iterrows():
            fifa_name = fifa_row['Name']
            fifa_first_initial = fifa_name[0].lower() if fifa_name else ""
            
            if (stats_first_initial == fifa_first_initial or 
                len(possible_fifa_matches) == 1):
                
                matches.append({
                    'stats_name': stats_name,
                    'fifa_name': fifa_name,
                    'last_name': stats_last_name,
                    'stats_index': stats_idx,
                    'fifa_index': fifa_idx,
                    'confidence': 'high' if stats_first_initial == fifa_first_initial else 'unique_lastname'
                })
                break  
    
    return matches

strict_matches = find_strict_matches()

if len(strict_matches) > 0:
    stats_indices = [match['stats_index'] for match in strict_matches]
    fifa_indices = [match['fifa_index'] for match in strict_matches]
    
    duplicate_stats = len(stats_indices) != len(set(stats_indices))
    duplicate_fifa = len(fifa_indices) != len(set(fifa_indices))
    
    if duplicate_stats or duplicate_fifa:
        seen_stats = set()
        seen_fifa = set()
        unique_matches = []
        
        for match in strict_matches:
            if (match['stats_index'] not in seen_stats and 
                match['fifa_index'] not in seen_fifa):
                
                seen_stats.add(match['stats_index'])
                seen_fifa.add(match['fifa_index'])
                unique_matches.append(match)
        
        strict_matches = unique_matches
    
    stats_indices = [match['stats_index'] for match in strict_matches]
    fifa_indices = [match['fifa_index'] for match in strict_matches]
    
    final_player_stats = player_stats.loc[stats_indices].drop('last_name_temp', axis=1)
    final_fifa_data = fifa_data.loc[fifa_indices].drop('last_name_temp', axis=1)
    
    if len(final_player_stats) != len(final_fifa_data):
        stats_players = set(final_player_stats['Player'])
        fifa_last_names = set(final_fifa_data['Name'].apply(get_last_name))
        
        problem_players = []
        for player in stats_players:
            if get_last_name(player) not in fifa_last_names:
                problem_players.append(player)
    
    validation_data = []
    for match in strict_matches:
        stats_player = final_player_stats[final_player_stats['Player'] == match['stats_name']].iloc[0]
        fifa_player = final_fifa_data[final_fifa_data['Name'] == match['fifa_name']].iloc[0]
        
        validation_data.append({
            'Stats_Player': match['stats_name'],
            'FIFA_Player': match['fifa_name'], 
            'Last_Name': match['last_name'],
            'Stats_Squad': stats_player['Squad'] if 'Squad' in stats_player else 'N/A',
            'FIFA_Club': fifa_player['Club'] if 'Club' in fifa_player else 'N/A',
            'FIFA_Overall': fifa_player['Overall'],
            'Confidence': match['confidence'],
            'VALIDIERT': '✅' 
        })
    
    validation_df = pd.DataFrame(validation_data)
    
    final_player_stats.to_csv('strict_player_stats.csv', index=False)

    final_fifa_data.to_csv('strict_fifa_data.csv', index=False)

    validation_df.to_csv('strict_validation.csv', index=False)

    stats_last_names = set(final_player_stats['Player'].apply(get_last_name))
    fifa_last_names = set(final_fifa_data['Name'].apply(get_last_name))
    
    missing_in_fifa = stats_last_names - fifa_last_names
    missing_in_stats = fifa_last_names - stats_last_names

player_stats = player_stats.drop('last_name_temp', axis=1)
fifa_data = fifa_data.drop('last_name_temp', axis=1)